# Data Cleaning

The Zillow datasets (home prices and rent) are currently in wide format with one column per month and contain data for all metropolitan areas in the United States.

In this section, I will:
- Filter the data to the San Jose MSA
- Remove unnecessary columns
- Reshape the data into long time-series format
- Prepare it for merging with income data

Income Dataset already fullly cleaned and merged. 

In [25]:
import pandas as pd
import numpy as np

rent = pd.read_csv('../data/interim/sj_rent_monthly.csv')
home_price = pd.read_csv('../data/interim/sj_home_prices_monthly.csv')

## Data Filtering

The Zillow datasets were filtered to retain only the San Jose, CA metropolitan area (MSA). Although this results in a single row per dataset, each row contains over 100 monthly observations representing aggregated housing values across the entire region. These values reflect metro-level housing trends rather than individual property prices. The data will be reshaped into time-series format for further analysis.

In [43]:
sj_rent = rent[
    (rent["RegionName"] == "San Jose, CA") &
    (rent["RegionType"] == "msa")
].copy()

sj_rent.shape


(1, 137)

In [58]:
sj_home = home_price[
    (home_price["RegionName"] == "San Jose, CA") &
    (home_price["RegionType"] == "msa")
].copy()

sj_home


,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
35,395059,36,"San Jose, CA",msa,CA,352038.848525,353853.226097,356586.552721,363988.597534,371749.188018,...,1.560681e+06,1.548911e+06,1.532506e+06,1.515530e+06,1.504037e+06,1.500369e+06,1.505465e+06,1.515774e+06,1.530369e+06,1.543938e+06


## Reshaping Data to Time-Series Format

The Zillow datasets are originally structured in wide format, where each row represents a region and each column represents a monthly observation. While this format is suitable for storage, it is not ideal for time-series analysis or merging with other datasets.

To prepare the data for analysis, the San Jose MSA rent and home price data are reshaped from wide format to long format using a melt operation. This transformation converts monthly columns into a single `date` column and a corresponding value column (`rent` or `home_price`).

After reshaping, each row represents one monthly observation for the San Jose metropolitan area. This time-series structure allows for chronological sorting, merging with income data, calculation of growth rates, and regression or predictive modeling.

This step ensures the dataset is properly structured for further analysis.


In [54]:
meta_cols = ["RegionID", "SizeRank", "RegionName", "RegionType", "StateName"]
date_cols = [c for c in sj_rent.columns if c not in meta_cols]
sj_rent_long = sj_rent.melt(
    id_vars=meta_cols,
    value_vars=date_cols,
    var_name="date",
    value_name="rent"
)

sj_rent_long["date"] = pd.to_datetime(sj_rent_long["date"])
sj_rent_long = sj_rent_long.sort_values("date").reset_index(drop=True)

sj_rent_long.head()


,RegionID,SizeRank,RegionName,RegionType,StateName,date,rent
0,395059,36,"San Jose, CA",msa,CA,2015-01-31,2442.261478
1,395059,36,"San Jose, CA",msa,CA,2015-02-28,2451.852475
2,395059,36,"San Jose, CA",msa,CA,2015-03-31,2471.268835
3,395059,36,"San Jose, CA",msa,CA,2015-04-30,2507.208362
4,395059,36,"San Jose, CA",msa,CA,2015-05-31,2565.092154


In [55]:
date_cols_home = [c for c in sj_home.columns if c not in meta_cols]

sj_home_long = sj_home.melt(
    id_vars=meta_cols,
    value_vars=date_cols_home,
    var_name="date",
    value_name="home_price"
)

sj_home_long["date"] = pd.to_datetime(sj_home_long["date"])
sj_home_long = sj_home_long.sort_values("date").reset_index(drop=True)

sj_rent_long

,RegionID,SizeRank,RegionName,RegionType,StateName,date,rent
0,395059,36,"San Jose, CA",msa,CA,2015-01-31,2442.261478
1,395059,36,"San Jose, CA",msa,CA,2015-02-28,2451.852475
2,395059,36,"San Jose, CA",msa,CA,2015-03-31,2471.268835
3,395059,36,"San Jose, CA",msa,CA,2015-04-30,2507.208362
4,395059,36,"San Jose, CA",msa,CA,2015-05-31,2565.092154
...,...,...,...,...,...,...,...
127,395059,36,"San Jose, CA",msa,CA,2025-08-31,3413.660781
128,395059,36,"San Jose, CA",msa,CA,2025-09-30,3412.484615
129,395059,36,"San Jose, CA",msa,CA,2025-10-31,3403.284032
130,395059,36,"San Jose, CA",msa,CA,2025-11-30,3395.052701


In [59]:
sj_housing = pd.merge(
    sj_rent_long[["date", "rent"]],
    sj_home_long[["date", "home_price"]],
    on="date",
    how="inner"
)

sj_housing.head()

,date,rent,home_price
0,2015-01-31,2442.261478,757953.079907
1,2015-02-28,2451.852475,767196.560902
2,2015-03-31,2471.268835,772689.807192
3,2015-04-30,2507.208362,775291.415394
4,2015-05-31,2565.092154,777993.176600


**Dropping Missing Values**

In this case, we wanted to keep every value so we decided to interpolate the one missing value. 

In [63]:
sj_housing.isna().sum()

date          0
rent          0
home_price    1
dtype: int64

In [69]:
sj_housing["home_price"] = sj_housing["home_price"].interpolate()
sj_housing.isna().sum()

date          0
rent          0
home_price    0
dtype: int64